<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/FI830_HW8_S24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.stats as st
from scipy.optimize import newton
from tqdm import tqdm

# Interest Rate Derivatives

Assume the Vasicek model:

$$
dr_t=\alpha(\beta-r_t)dt+\gamma\,dW_t
$$

with (risk-neutral) parameters $\alpha=0.3,\ \beta=4\%,\ \gamma=0.02,$ and $r_0=5.4\%$ reflecting the relevant yield curve.

In [ ]:
# Parameters
alpha = 0.3; beta = 0.04; gamma = 0.02; r0 = 0.054
T = 5; n_coup = 10 # number of coupons/interests is 10 since paid semi-annually

## (a) Five-year Treasury Note
Under a constant yield curve, the yield of a 5-year T-note is a constant, $y$, such that
$$
\sum_{k=1}^{10} \frac{y}{2}\,e^{-rk/2}+e^{-5r} = 1
$$

In [ ]:
def pv_coupon(y,r,T,n): # note that here y is the semiannual yield
  val = 0
  for k in range(1,n+1):
    val += y*np.exp(-r*k*(T/n))
  return val

yield_const_semi = newton(lambda y: pv_coupon(y,r0,T,n_coup)+np.exp(-T*r0)-1,0.05)
yield_const_ann = yield_const_semi*2
yield_const_ann

0.05473560552697876

Under the Vasicek model, the price of a coupon-paying bond with face value $F$ and maturity $T$ at time $t$ is
$$
P_c(F,t,T) = F*P_0(t,T) + \sum_{k=1}^n\,C_kP_0(t,T_k),
$$
where $P_0(t,T)$ is the price of a zero-coupon bond paying $1$ at maturity $T.$

We have
$$
P_0(t,T) = A(t,T)e^{-r_t B(t,T)}
$$

where
$$
B(t,T) = \frac{1-e^{-\alpha(T-t)}}{\alpha}
$$
and
$$
A(t,T) = \text{exp}\left\{\left(\beta-\frac{\gamma^2}{2\alpha^2}\right)\left(B(t,T)-(T-t)\right)-\frac{\gamma^2}{4\alpha}B^2(t,T)\right\}
$$

In [ ]:
def P0_Vasicek(rt,t,T):
  B_t_T = (1-np.exp(-alpha*(T-t)))/alpha
  A_t_T = np.exp((beta-gamma**2/(2*alpha**2))*(B_t_T-(T-t))-(gamma**2/(4*alpha))*B_t_T**2)
  return A_t_T*np.exp(-rt*B_t_T)

def Pc_Vasicek(F,rt,y,t,T,n): # here C_k=F*y and y is again semiannual yield
  val = 0
  for k in range(1+int(t*n/T),n+1):
    val += F*y*P0_Vasicek(rt,t,k*T/n)
  return val + F*P0_Vasicek(rt,t,T)

In [ ]:
yield_vasicek = newton(lambda y: Pc_Vasicek(1,r0,y,0,T,n_coup)-1,0.05) # WLOG, let F=1
yield_vasicek*2

0.04741123923398437

## (b)1-Year Forward Contract on the 5-Year T-Note
We use Monte Carlo simulation to get interest rate in one year. Let $h$ be the step size. An Euler discretization gives
$$
\begin{align*}
r_{t+1} &= r_t + \Delta r_t \\
&= r_t + \alpha(\beta-r_t)h+\gamma(W_{t+1}-W_t) \\
&= r_t + \alpha(\beta-r_t)h+\gamma\sqrt{h}Z_{t}
\end{align*}
$$
where $Z_{t}\sim N(0,1).$

In [ ]:
# need to know r_1
F = 100000; T_frwd = 1; N = 10000; h = 0.001
np.random.seed(12)
prices = [0] * N
for n in tqdm(range(N)):
  k = int(T_frwd/h+1)
  r_t = [r0] * k
  Z = np.random.normal(0,1,k-1)
  for i in range(1,k):
    r_t[i] = r_t[i-1] + alpha*(beta-r_t[i-1])*h + gamma*np.sqrt(h)*Z[i-1]
  prices[n] = Pc_Vasicek(F,r_t[-1],yield_vasicek,T_frwd,T,n_coup)
forward_p = np.average(prices)
forward_p # value of the 5-year T-note in one year

100%|██████████| 10000/10000 [00:24<00:00, 407.36it/s]


100515.10344149271

$\text{Forward price} = \text{Spot price} + \text{Cost of Carry}.$ More precisely,
\begin{align*}
F(t,T_{frwd}) = (P_c(F,t,T_{bond}) - P_c(F,t,T_{frwd}) + F*P_0(t,T_{frwd}))\ e^{\int_t^{T_{frwd}}r_t\,dt}
\end{align*}

In [ ]:
# verify if the value of the 5-year T-note in one year equals the price from the equation
F = 100000; T_frwd = 1; N = 10000; h = 0.001
np.random.seed(12)
prices = [0] * N; prices2 = [0] * N; prices3 = [0] * N
for n in tqdm(range(N)):
  k = int(T_frwd/h+1)
  r_t = [r0] * k
  Z = np.random.normal(0,1,k-1)
  for i in range(1,k):
    r_t[i] = r_t[i-1] + alpha*(beta-r_t[i-1])*h + gamma*np.sqrt(h)*Z[i-1]
  r_hat = (0.5*r_t[0] + sum(r_t[1:-1]) + 0.5*r_t[-1]) * h
  r_hat2 = (0.5*r_t[int(T_frwd/2/h)]+sum(r_t[int(T_frwd/2/h+1):-1])+0.5*r_t[-1]) * h
  prices[n] = Pc_Vasicek(F,r_t[-1],yield_vasicek,T_frwd,T,n_coup)
  prices2[n] = (Pc_Vasicek(F,r0,yield_vasicek,0,T,n_coup)-Pc_Vasicek(F,r0,yield_vasicek,0,T_frwd,T_frwd*2)+F*P0_Vasicek(r0,0,T_frwd))*np.exp(r_hat)
  prices3[n] = Pc_Vasicek(F,r0,yield_vasicek,0,T,n_coup)*np.exp(r_hat) - F*yield_vasicek - F*yield_vasicek*np.exp(r_hat2)
np.average(prices), np.average(prices2), np.average(prices3) # all similar

KeyboardInterrupt: 

## (c) European Call on the 5-Year T-Note Expiring in 1 Year
The value (at time $t$) of a European Call option on an $S$-year T-note with an option maturity of $T$ years and strike price of $K$ is
$$
e^{-\int_t^T r_s\,ds}\text{max}\{P_c(F,T,S)-K,0\}
$$

In [ ]:
K = forward_p; T_call = 1

In [ ]:
def MCVesicekCall(F,K,Tc,Tb,n_coup,h,N):
  np.random.seed(12)
  callP = [0] * N
  k = int(Tc/h+1)
  for n in range(N):
    r_t = [r0] * k
    Z = np.random.normal(0,1,k-1)
    for i in range(1,k):
      r_t[i] = r_t[i-1] + alpha*(beta-r_t[i-1])*h + gamma*np.sqrt(h)*Z[i-1]
    r_hat = (0.5*r_t[0] + sum(r_t[1:-1]) + 0.5*r_t[-1]) * h
    bondP = Pc_Vasicek(F,r_t[-1],yield_vasicek,Tc,Tb,n_coup)
    callP[n] = max(bondP-K,0) * np.exp(-r_hat)
  return np.average(callP), np.std(callP)

In [ ]:
N = 10000; h = 0.001
C0 = MCVesicekCall(F,K,T_call,T,n_coup,h,N)[0]
print("The Bond option price is $%.3f." % C0)

The Bond option price is $1449.242.


In [ ]:
# 95% CI
C0_est, C0_std = MCVesicekCall(F,K,T_call,T,n_coup,h,N)
C0_lb, C0_ub = C0_est-st.norm.ppf(0.975)*C0_std/np.sqrt(N), C0_est+st.norm.ppf(0.975)*C0_std/np.sqrt(N)
(C0_lb,C0_ub)

(1406.4129793566324, 1492.071147822547)

In [ ]:
C0_std

2185.1975123414327